In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
공개 데이터셋에서 학습, 테스트 데이터 내려받기.
"""

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 10299725.19it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 104482.98it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 7748215.74it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
"""
훈련, 테스트데이터 배치로 로드하기
"""
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size)

for X, y  in test_dataloader:
  print("Shape of X [N, C, H, W]:", X.shape)
  print("Shape of y:", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
"""
사용 가능한 장치 확인하기
"""
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)
print("device:", device)


device: cuda


In [7]:
"""
신경망 모델 만들기
"""

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    # nn.Flatten: 다차원 입력을 1차원으로 변환해줌. ex) [4, 3, 28, 28] => [4, 2352]
    self.flatten = nn.Flatten()

    # nn.Linear: Affine 레이어
    # nn.Sequential: 신경망 체인 만들기.
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10)
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print("model:", model)

model: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
"""
모델 학습을 위해 손실함수(loss function)와 옵티마이저(optimizer) 준비하기.
"""

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

print("loss function:", loss_fn)
print("optimizer:", optimizer)

loss function: CrossEntropyLoss()
optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
"""
각 학습데이터셋에 대해 추론 후
역전파를 수행해 각 신경망의 미분값을 얻고
이를 바탕으로 경사하강법을 사용해 매개변수를 조절한다(SGD)
"""

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
"""
테스트 데이터를 통해 추론하고 추론 결과를 정답지와 비교 후 정답률을 계산하여 출력한다.
"""

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
"""
신경망을 훈련 후 테스트하여 정확도를 파악한다.
"""

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.247496  [   64/60000]
loss: 0.374708  [ 6464/60000]
loss: 0.235761  [12864/60000]
loss: 0.417732  [19264/60000]
loss: 0.322195  [25664/60000]
loss: 0.369642  [32064/60000]
loss: 0.357188  [38464/60000]
loss: 0.554614  [44864/60000]
loss: 0.474774  [51264/60000]
loss: 0.364034  [57664/60000]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.407172 

Epoch 2
-------------------------------
loss: 0.238864  [   64/60000]
loss: 0.373325  [ 6464/60000]
loss: 0.234540  [12864/60000]
loss: 0.416216  [19264/60000]
loss: 0.321734  [25664/60000]
loss: 0.369122  [32064/60000]
loss: 0.357015  [38464/60000]
loss: 0.553489  [44864/60000]
loss: 0.473667  [51264/60000]
loss: 0.363594  [57664/60000]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.406664 

Epoch 3
-------------------------------
loss: 0.238370  [   64/60000]
loss: 0.372569  [ 6464/60000]
loss: 0.234077  [12864/60000]
loss: 0.415139  [19264/60000]
loss: 0.321021  [25664/60000]
loss: 0.368545  [32064/600

In [24]:
"""
모델 저장하기
"""
torch.save(model.state_dict(), "fashion_mnist_model_v1.pth")

model saved


In [21]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("fashion_mnist_model_v1.pth"))

Test Error: 
 Accuracy: 8.9%, Avg loss: 2.306775 

Test Error: 
 Accuracy: 85.8%, Avg loss: 0.402665 

